In [ ]:
import pandas as pd
import psycopg2 as pg
import streamlit as st
import warnings
from warnings import filterwarnings

filterwarnings("ignore", category=UserWarning,
               message=".*pandas only supports SQLAlchemy connectable.*")

In [ ]:
db_credenciais = st.secrets['postgresql']

host = db_credenciais['host']
port = db_credenciais['port']
user = db_credenciais['user']
password = db_credenciais['password']
dbname = db_credenciais['dbname']

with pg.connect(host=host, port=port, user=user, password=password, dbname=dbname) as conn:
    curr = conn.cursor()

    enem_2024 = pd.read_sql_query("""SELECT
                                            co_municipio_esc,
                                            AVG(nota_cn_ciencias_da_natureza) as media_cn,
                                            AVG(nota_ch_ciencias_humanas) as media_ch,
                                            AVG(nota_lc_linguagens_e_codigos) as media_lc,
                                            AVG(nota_mt_matematica) as media_mt,
                                            AVG(nota_redacao) as media_redacao
                                    FROM
                                        public.ed_enem_2024_resultados
                                    WHERE
                                        co_municipio_esc IS NOT NULL
                                    GROUP BY
                                        co_municipio_esc
                                            """, conn)

    pib_municipios = pd.read_sql_query("""SELECT
                                                codigo_municipio_dv,
                                                ano_pib,
                                                vl_pib,
                                                vl_pib_per_capta
                                        FROM
                                                public.pib_municipios
                                                """, conn)

    censo = pd.read_sql_query("""SELECT
                                    "CO_MUNICIPIO",
                                    "TOTAL",
                                    "IDADE"
                                FROM
                                    public."Censo_20222_Populacao_Idade_Sexo"
                                WHERE
                                    "IDADE" BETWEEN 15 AND 19
                                    """, conn)

    municipio = pd.read_sql_query("""SELECT
                                            nome_municipio,
                                            codigo_municipio_dv
                                    FROM
                                            public.municipio
                                            """, conn)

    uf = pd.read_sql_query("""SELECT
                                cd_uf,
                                sigla_uf,
                                nome_uf
                            FROM
                                public.unidade_federacao
                        """, conn)

    df_temp = pd.merge(enem_2024, pib_municipios, left_on='co_municipio_esc',
                       right_on='codigo_municipio_dv', how='left')

    df = pd.merge(df_temp, censo, left_on='co_municipio_esc',
                  right_on='CO_MUNICIPIO', how='left')

243740.99016189575
(190654207, 13)
